In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn #module to implement neural network
import torch.nn.parallel #module for parallel computation
import torch.optim as optim #optimizer
import torch.utils.data  #stochastic gradient descent
from torch.autograd import Variable